# Experimenting with BertDistractor

In [1]:
import sys, os
import torch
import bbrsa
sys.path.append(os.path.abspath(bbrsa.ONMT_DIR))

from bbrsa.distractors import BertDistractor
from bbrsa.bbrsa import ONMTSummaryRSA
from bbrsa.summarizers import ONMTSummarizer

s0 = ONMTSummarizer(config_path='onmt_configs/giga.yml')
# pragmatics = BasicPragmatics(alpha=1)
distr = BertDistractor(batch_size=s0.default_batch_size)

Building ONMT translator with configs from onmt_configs/giga.yml
Finished building ONMT translator.
Initializing Bert models for distractor
Finished initializing


## Explanation

The following are some examples of what the Bert Distractor generates for now.

Currently the method is to get a heuristic measure of saliency for each word from BERT's outputs. Then I pick 5 most salient words in the sentence according to that measure (excluding the beginning `[CLS]` and ending `[SEP]` tokens), mask all words using the `[MASK]` token, and let BERT predict a word for these masked positions as distractors.

Currently there are two possible heuristic measures: 
  - the summed up and normalized attention given by the layer-0 transformer in BERT, this one is by default.
  - the negative log probability for each source word given by BERT's language model, with the source article with out any mask as input. Use this heuristic by specifying `method='unmasked_surprisal'` in the `generate()` method of the distractor
    
Also I force the distractor to choose a different word if BERT predicts the same word as before for a masked position. This can be disabled in specifying the keyword argument `ensure_different=False` in the `generate()` method.

In [2]:
src = ['police arrested five anti-nuclear protesters thursday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .']
print(distr.generate(src))
print(distr.generate(src, method='unmasked_surprisal'))

['police arrested five anti-nuclear protesters thursday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .', 'police detained five anti-nuclear activists thursday after they attempted to disrupt loading of a french nuclear research and supply vessel , a spokesman for the group said .']
['police arrested five anti-nuclear protesters thursday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .', 'authorities arrested five anti-nuclear protesters , after they objected to the loading of a french antarctic research and supply ship , a spokesman for the protesters said .']


__SRC__: police arrested five anti-nuclear protesters thursday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .

__attn distractor__: police ___detained___ five anti-nuclear ___activists___ thursday after they ___attempted____ to disrupt loading of a french ___nuclear___ research and supply vessel , a spokesman for the ___group___ said .

__unmasked surprisal__: ___authorities___ arrested five anti-nuclear protesters ___,___ after they ___objected___ to ___the___ loading of a french antarctic research and supply ___ship___ , a spokesman for the protesters said .


In [3]:
src = ['japan \'s toyota team europe were banned from the world rally championship for one year here on friday in a crushing ruling by the world council of the international automobile federation -lrb- fia -rrb- .']
print(distr.generate(src))
print(distr.generate(src, method='unmasked_surprisal'))

["japan 's toyota team europe were banned from the world rally championship for one year here on friday in a crushing ruling by the world council of the international automobile federation -lrb- fia -rrb- .", "japan 's and team europe were suspended from the world championship cup for one year here on friday in a special ruling by the world council of the international automobile federation -lrb- fia -rrb- ."]
["japan 's toyota team europe were banned from the world rally championship for one year here on friday in a crushing ruling by the world council of the international automobile federation -lrb- fia -rrb- .", "ford 's ford team drivers were banned from the world rally championship for one year here on friday in a controversial ruling by the world council of the international automobile federation -lrb- fia sco ."]


__src__ japan 's toyota team europe were banned from the world rally championship for one year here on friday in a crushing ruling by the world council of the international automobile federation -lrb- fia -rrb- .

__attn distractor__: japan 's ___and___ team europe were ___suspended___ from the world ___championship cup___ for one year here on friday in a ___special___ ruling by the world council of the international automobile federation -lrb- fia -rrb- .

__unmasked surprisal__: ___ford___ 's ___ford___ team ___drivers___ were banned from the world rally championship for one year here on friday in a ___controversial___ ruling by the world council of the international automobile federation -lrb- fia ___sco___ .

In [4]:
src = ['the bank of japan appealed to financial markets to remain calm friday following the us decision to order daiwa bank ltd. to close its us operations .']
print(distr.generate(src))
print(distr.generate(src, method='unmasked_surprisal'))

['the bank of japan appealed to financial markets to remain calm friday following the us decision to order daiwa bank ltd. to close its us operations .', 'the bank of america agreed to financial reasons to remain on friday following the us decision to order daiwa bank inc. to close its us operations .']
['the bank of japan appealed to financial markets to remain calm friday following the us decision to order daiwa bank ltd. to close its us operations .', 'central bank of japan appealed to financial markets to keep calm , following the us decision to order iwa bank corp. to close its us operations .']


__SRC__: the bank of japan appealed to financial markets to remain calm friday following the us decision to order daiwa bank ltd. to close its us operations .

__attn distractor__: the bank of ___america___ agreed to financial ___reasons___ to remain ___on___ friday following the us decision to order daiwa bank ___inc.___ to close its us operations .

__unmasked__: ___central___ bank of japan appealed to financial markets to ___keep___ calm ___,___ following the us decision to order ___iwa___ bank ___corp.___ to close its us operations .

In [5]:
src = ['factory orders for manufactured goods rose #.# percent in september , the commerce department \'s director and p.v. announced together on thursday .']
print(distr.generate(src))
print(distr.generate(src, method='unmasked_surprisal'))

["factory orders for manufactured goods rose #.# percent in september , the commerce department 's director and p.v. announced together on thursday .", "the orders for the s rose #.# percent in september , the war department 's director and p.v. announced together on facebook ."]
["factory orders for manufactured goods rose #.# percent in september , the commerce department 's director and p.v. announced together on thursday .", "total orders for manufactured products products #.# percent in september , the commerce department 's director and p.i. announced together on facebook ."]


__src__: factory orders for manufactured goods rose #.# percent in september , the commerce department 's director and p.v. announced together on thursday .

__attn__: ___the___ orders for the ___s___ rose #.# percent in september , the ___war___ department 's director and p.v. announced together on ___facebook___ .

__unmask__: ___total___ orders for manufactured ___products products___ #.# percent in september , the commerce department 's director and ___p.i.___ announced together on ___facebook___ .